In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from network_builder import*
import constants 
import latent_semantic_analysis

# from recommender.core import network_builder

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/huynhhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/huynhhao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
companies_df = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\companies.csv')
jobs_df = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\jobs.csv')
cv = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\cvdata\ResumeDataSet.csv', encoding = 'utf-8')

In [39]:
# transform companies_df and jobs_df into dicts
employers_data = {}
jobs_data = {}
for i, row in companies_df.iterrows():
    employer_data = {'company_name': row['company_name'],
                   'average_rating': row['average_rating'],
                   'num_review': row['num_review'],
                   'city': row['city'],
                   'type': row['type'],
                   'num_employee': row['num_employee'],
                   'country': row['country'],
                   'working_day': row['working_day'],
                   'OT': row['OT'],
                   'overview': row['overview'],
                   'expertise': row['expertise'],
                   'benifit': row['benifit'],
                   'logo_link': row['logo_link']}
    
    employers_data[row['company_id']] = employer_data
    
for i, row in jobs_df.iterrows():
    job_data = {'company_id': row['company_id'],
               'job_name': row['job_name'],
               'taglist': row['taglist'],
               'location': row['location'], 
               'three_reasons': row['three_reasons'],
               'description': row['description']}
    jobs_data[row['job_id']] = job_data
    

In [68]:
G = nx.MultiDiGraph(name = 'Jobs graph')
# first add all employer nodes to the network and its data
for employer_id, employer_data in employers_data.items():
    G.add_node(employer_id, node_type = 'employer', **employer_data)
    
# add all job nodes and the bidirectional edges from job node to employer node
for job_id, job_data in jobs_data.items():
    G.add_node(job_id, node_type = 'job', **job_data)
    # add two edges between job and its employers\
    G.add_edge(job_id, job_data['company_id'], weight = 1)
    G.add_edge(job_data['company_id'], job_id, weight = 1)

In [74]:
G['kms-technology']

AdjacencyView({'kms-technology:jrsr_qa_engineer_kms_labs_bonus': {0: {'weight': 1}}, 'kms-technology:engineering_manager_bonus': {0: {'weight': 1}}, 'kms-technology:fullstack_mobile_mobilenodejs_kobiton': {0: {'weight': 1}}, 'kms-technology:jrsrprincipal_java_developer_bonus': {0: {'weight': 1}}, 'kms-technology:product_manager_kms_labs_bonus': {0: {'weight': 1}}, 'kms-technology:sr_it_business_analyst_english_bonus': {0: {'weight': 1}}, 'kms-technology:fullstack_dev_reactjsnodejs_kobiton': {0: {'weight': 1}}, 'kms-technology:senior_ruby_on_rails_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:senior_data_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:srjr_fullstack_nodejsreactjs_bonus': {0: {'weight': 1}}, 'kms-technology:juniorsenior_test_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:jrsrprincipal_net_developer_bonus': {0: {'weight': 1}}})

In [3]:
nb = network_builder.NetworkBuilder(constants.EMPLOYERS_DATA_PATH,
                                    constants.JOBS_DATA_PATH, constants.CV_DATAPATH )

In [5]:
nb.build()

2021-11-26 06:47:00,391 - Start building the master Graph...
2021-11-26 06:47:00,418 - Master Graph is built.
2021-11-26 06:47:00,419 - Creating LSA comparer
2021-11-26 06:47:00,420 - Loading LSA comparer from /home/huynhhao/Desktop/job_recommender/data/network_data/lsa.pkl
2021-11-26 06:47:00,518 - Adding relations edges...


In [8]:
len(nb.G.nodes)

4414

In [31]:
nb.lsa.preprocess_text(nb.G.nodes['kms-technology']['overview'])

'established km technology usbased engineering service company development center vietnam km technology trusted international client superior quality product expertise vietnamese engineer km build successfully launch software company internal startup _unknown_ km lab _unknown_ company include qasymphony kobiton katalon _unknown_ brand km solution company serf asia pacific region offering technology solution consulting bringing advanced latest technology asian market km committed making _unknown_ social impact partnering _unknown_ organization give _unknown_ student fair better employment opportunity company regularly _unknown_ university participates training activity specialized event sponsor scholarship program vietnam company received many prestigious industry award named one greatest place work vietnam asia u many year row'

In [3]:
with open(constants.NETWORK_BUILDER_SAVE_PATH, 'rb') as f:
    network = pickle.load(f)

In [4]:
len(network.G['kms-technology'])

32

In [6]:
network.G['kms-technology']

AdjacencyView({'kms-technology:jrsr_qa_engineer_kms_labs_bonus': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:engineering_manager_bonus': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:fullstack_mobile_mobilenodejs_kobiton': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:jrsrprincipal_java_developer_bonus': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:product_manager_kms_labs_bonus': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:sr_it_business_analyst_english_bonus': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:fullstack_dev_reactjsnodejs_kobiton': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:senior_ruby_on_rails_engineer_bonus': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:senior_data_engineer_bonus': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:srjr_fullstack_nodejsreactjs_bonus': {0: {'weight': 1, 'edge_type': 'posted'}}, 'kms-technology:juniorsenior_test_engineer

In [8]:
network.G.nodes['candidate-0']

{'node_type': 'candidate',
 'expertise': 'Data Science',
 'resume': 'Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\n\r\nData Science Assurance Associate \r\n\r\nData Science Assurance Associate - Ernst & Young LLP\r\nSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details